Ερώτημα 2
=========

In [1]:
import pandas as pd

data = pd.read_csv("./data/onion-or-not.csv")
print("Diastaseis {}".format(data.shape))
data.head()

Diastaseis (24000, 2)


,text,label
0,Entire Facebook Staff Laughs As Man Tightens P...,1
1,Muslim Woman Denied Soda Can for Fear She Coul...,0
2,Bold Move: Hulu Has Announced That They’re Gon...,1
3,Despondent Jeff Bezos Realizes He’ll Have To W...,1
4,"For men looking for great single women, online...",1


Δημιουργία διανύσματος λέξεων

In [2]:
original_words = [text.split(" ") for text in data["text"]]

Επεξεργασία διανύσματος λέξεων

In [3]:
from nltk.stem import PorterStemmer

ps = PorterStemmer()
processed_words = []

# Stemming
for sentence in original_words:
    processed_words.append([ps.stem(word) for word in sentence])

In [4]:
import nltk
from nltk.corpus import stopwords

# Stop words removal
try:
    stop_words = set(stopwords.words('english'))
except:
    nltk.download('stopwords')
    stop_words = set(stopwords.words('english'))

for i in range(len(processed_words)):
    j = 0
    while j < len(processed_words[i]):
        word = processed_words[i][j]
        if word in stop_words:
            processed_words[i].pop(j)
        j += 1

Υπολογισμός tf-idf μετρικής

In [5]:
import numpy as np
from joblib import Parallel, delayed
import multiprocessing

# N = len(processed_words)
N = 1000
sentence_lengths = [len(processed_words[i]) for i in range(N)]
M = max(sentence_lengths)
words_num = sum(sentence_lengths)

# Custom code - Not working well
# f = np.zeros([N, M])
# for i in range(N):
#     sentence = processed_words[i]
#     for j in range(len(sentence)):
#         word = sentence[j]
#         f[i][j] = sentence.count(word) / words_num
# tf = f/f.max()


# def countApperances(i):
#     dfi = []
#     # dfi = Parallel(n_jobs=num_cores)(delayed(countWordApperances)(i,j) for j in range(len(processed_words[i])))
#     dfi = [countWordApperances(i,j) for j in range(len(processed_words[i]))]
#     # print(dfi)
#     dfi = np.array(dfi)
#     return dfi

# def countWordApperances(i,j):
#     word = processed_words[i][j]
#     dfij = np.array([word in sentence for sentence in processed_words]).sum()
#     return dfij

# num_cores = multiprocessing.cpu_count()
# # df = Parallel(n_jobs=num_cores, verbose=10)(delayed(countApperances)(i) for i in range(N))
# df = [countApperances(i) for i in range(N)]
# df = np.array(df)

# idf = np.zeros([N,M])
# idf = N/df
# idf = np.array([np.log2(idf[i]) for i in range(len(idf))])


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
X = tfidf.fit_transform(data["text"].to_numpy())
Y = data["label"].to_numpy()

In [8]:
print(X[0, tfidf.vocabulary_['facebook']])
print(X.shape)

0.24772077883846477
(24000, 24526)
